In [ ]:
!pip install PyPDF2 cohere langchain faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 521.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.7/249.7 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.7/407.7 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.7/296.7 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/5

In [ ]:
from google.colab import files
uploaded=files.upload()

Saving STOCKMARKETANALYSISUSINGMAPREDUCEANDPYSPARK.pdf to STOCKMARKETANALYSISUSINGMAPREDUCEANDPYSPARK.pdf


In [ ]:
import PyPDF2
pdf_file_path=next(iter(uploaded))
with open(pdf_file_path,'rb') as file:
  reader=PyPDF2.PdfReader(file)
  text=''
  for page_num in range(len(reader.pages)):
    text+=reader.pages[page_num].extract_text()
print(text[:500])


See discussions, st ats, and author pr ofiles f or this public ation at : https://www .researchgate.ne t/public ation/356776423
STOCK MARKET ANALYSIS
Article  · July 2021
CITATIONS
13READS
34,388
6 author s, including:
Marimuthu Muthuv el
Coimb atore Instit ute of T echnolog y
29 PUBLICA TIONS    192 CITATIONS    
SEE PROFILE
Palanisw amy V elvadivu
Coimb atore Instit ute of T echnolog y
12 PUBLICA TIONS    24 CITATIONS    
SEE PROFILE
All c ontent f ollo wing this p age was uplo aded b y Marimu


In [ ]:
import cohere
api_key='dP6A0GmmjwmbLmbhOFBfNvIaz4Di0Qh9JJleyLEI'
cohere_client=cohere.Client(api_key)
chunks=[text[i:i+500] for i in range (0,len(text),500)]

response=cohere_client.embed(
    model='embed-english-v2.0',
    texts=chunks
)

embeddings=response.embeddings
print(f"Generated{len(embeddings)} embeddings")

Generated84 embeddings


In [ ]:
import faiss
import numpy as np
embedding_dim=len(embeddings[0])
embedding_matrix=np.array(embeddings).astype('float32')
index=faiss.IndexFlatL2(embedding_dim)
index.add(embedding_matrix)
print(len(embedding_matrix))

84


In [ ]:
import numpy as np
query=input("Enter a Query : ")
query_embedding=cohere_client.embed(
    model='embed-english-v2.0',
    texts=[query]
).embeddings[0]
query_embedding=np.array(query_embedding).astype('float32').reshape(1,-1)
D, I=index.search(query_embedding,k=5)


Enter a Query : @International Research Journal of Modernization in Engineering, Technology and Science    which year


In [ ]:
if I is not None:
    # Print out the top 5 chunks (first 300 characters)
    for idx in I[0]:
        print(chunks[idx][:300])
else:
    print("No results found")

                                         e-ISSN: 2582 -5208 
International Research Journal of  Modernization  in Engineering   Technology    and  Science  
( Peer -Reviewed, Open Access, Fully Refereed International Journal )  
Volume:03/Issue:07/July -2021             Impact Factor - 5.354        
                                                                                                            e-ISSN: 2582 -5208 
International Research Journal of  Modernization  in Engineering   Technology    and  Science  
( Peer -Reviewed, Open Access, Fully Refereed International Journal )  
Volu
pen Access, Fully Refereed International Journal )  
Volume:03/Issue:07/July -2021             Impact Factor - 5.354                            www.irjmets.com   
www.irjmets.com                               @International Research Journal of Modernization in Engineering, Technology and Science  
 
                                          e-ISSN: 2582 -5208 
International Research Journal of  

In [ ]:
top_match=chunks[I[0][0]]
response=cohere_client.generate(
    model='command-xlarge',
    prompt=f"Given the following information: {top_match} \nAnswer the query: {query}",
    max_tokens=150,
    temperature=0.7

)
print("Generated response:",response.generations[0].text)

Generated response:  The query @International Research Journal of Modernization in Engineering, Technology and Science is linked to the ISSN/ISBN number provided, hence it was published starting in 2021. 
